# Plotting data from the Heliospheric MHD simulation
***

In this notebook, some of the tools for making basic plots from the 3D MHD simulation data is presented. We are going to assume that data from the twin CME example is available. If not, run it first, or select some other data that you have available.

First, import the usual suspects:

In [ ]:
import numpy as np
import datetime

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import euhforia.core.constants as constants

Let's load the data. This is done using the `euhforia.core.io` module:

In [ ]:
import euhforia.core.io

In [ ]:
data = euhforia.core.io.load_heliospheric_data("../../run/twin_cme_output/twin_cme_2015-06-22T12-5*")

Note that the function automatically merges distributed data (if a MPI run was made), so a merging of files is not needed to be performed as a separate step.

The basic slice plots are available in the `euhforia.plot.slice` module.

In [ ]:
import euhforia.plot.slice

## Equatorial plane plots

Let's plot the Heliographic equatorial plane:

In [ ]:
euhforia.plot.slice.equatorial(data, "vr", figsize=(15,6)); 

By default, the colorbar limits etc. are chosen automatically. Let's select to use the "citrus" colormap with appropriate limits:

In [ ]:
import euhforia.plot.colormap

In [ ]:
euhforia.plot.slice.equatorial(data, "vr", 
                               levels=np.linspace(200, 1600, 120),
                               cmap=euhforia.plot.colormap.citrus,
                               figsize=(15,6)); 

We can also opt to plot only a part of the domain:

In [ ]:
fig, ax, cax = euhforia.plot.slice.equatorial(data, "vr", 
                                              levels=np.linspace(200, 1600, 120),
                                              cmap=euhforia.plot.colormap.citrus,
                                              figsize=(15,6)); 
ax.set_xlim((0.0, 0.5));
ax.set_ylim((0.0, 0.5));

## Meridional plane plots

The meridional slices work in a similar way:

In [ ]:
fig, ax, cax = euhforia.plot.slice.meridional(data, "vr", 
                                              lon=45.0*np.pi/180.0, # Which plane to select
                                              levels=np.linspace(200, 1600, 120),
                                              cmap=euhforia.plot.colormap.citrus,
                                              figsize=(13, 8))
ax.set_xlim((0, 0.7));
ax.set_ylim((-0.5, 0.5));

Other variables can naturally also be plotted:

In [ ]:
fig, ax, cax = euhforia.plot.slice.meridional(data, "n", lon=45.0*np.pi/180.0,
                                              levels=np.linspace(0, 90, 20),
                                              figsize=(13, 8))

In [ ]:
fig, ax, cax = euhforia.plot.slice.meridional(data, "Blon", lon=45.0*np.pi/180.0,
                                              levels=np.linspace(-5, 5, 120),
                                              figsize=(13, 8))

## Adding planets and spacecraft

The satellite/planet positions can also be included. First, the objects need to be instantiated. We also import the definitions for the plot marker and colors for the planets and spacecraft.

In [ ]:
import euhforia.orbit
import euhforia.plot.marker

In [ ]:
mercury = euhforia.orbit.Mercury()
venus = euhforia.orbit.Venus()
earth = euhforia.orbit.Earth()
mars = euhforia.orbit.Mars()
sta = euhforia.orbit.STA()
stb = euhforia.orbit.STB()

heliospheric_objects = (mercury, venus, earth, mars, sta, stb)

Now their positions can be added

In [ ]:
fig, ax, cax = euhforia.plot.slice.equatorial(data, "vr", 
                                              levels=np.linspace(200, 1600, 120),
                                              cmap=euhforia.plot.colormap.citrus,
                                              figsize=(15,8)); 

for obj in heliospheric_objects:
    
    # HEEQ position in spherical coords
    r, lat, lon = obj.position(data.datetime)
    
    # Don't plot if object is further than outer boundary of data
    if r > max(data.grid.domain.r):
        continue
        
    # Convert to Cartesian equatorial coords
    ax.plot(r*np.cos(lon)/constants.astronomical_unit, 
            r*np.sin(lon)/constants.astronomical_unit, 
            'o', label=obj.name, markersize=8, 
            marker=euhforia.plot.marker.style[obj.name]['marker'], 
            color=euhforia.plot.marker.style[obj.name]['color'])

ax.legend(loc='lower center', bbox_to_anchor=(0.5, -0.2), ncol=4, numpoints=1, fancybox=True)

## Plotting the equatorial plane and a meridional plane in the same figure

Often, one is interested in plotting the equatorial and a meridional slice at once. This can be achieved as follows:

In [ ]:
# Define the meridional slice to plot
meridional_slice = sta.position(data.datetime)[2]

In [ ]:
fig = plt.figure(figsize=(10, 5.5))

gs  = matplotlib.gridspec.GridSpec(1, 2, width_ratios=[1.38, 1.0])

ax1 = plt.subplot(gs[0])
ax2 = plt.subplot(gs[1])

#
# Equatorial plot
#
fig, ax1, cax = euhforia.plot.slice.equatorial(data, "vr", fig=fig, ax=ax1,
                                              levels=np.linspace(200, 1600, 120),
                                              cmap=euhforia.plot.colormap.citrus, colorbar=False,
                                              write_date=True);

#
# Meridional plot
#
fig, ax2, cax = euhforia.plot.slice.meridional(data, "vr", fig=fig, ax=ax2,
                                              lon=meridional_slice,#45.0*np.pi/180.0, # Which plane to select
                                              levels=np.linspace(200, 1600, 120),
                                              cmap=euhforia.plot.colormap.citrus, colorbar=False,
                                              write_date=False);

ax2.set_xlim((0.0, 1.0));

#
# Add colorbar
#
cbar = fig.colorbar(cax, ax=ax2, pad=0.25,
                    ticks=np.linspace(200, 1600, 8),
                    fraction=0.06, extend="both")

variable = "vr"
   
cbar.ax.set_ylabel(str(variable) + "  [" + data.units[variable] + "]", labelpad=10.0)

#
# Add planets and SC
#
for obj in heliospheric_objects:
    
    r, lat, lon = obj.position(data.datetime)
    
    # Don't plot if object is further than outer boundary of data
    if r > max(data.grid.domain.r):
        continue
    
    ax1.plot(r*np.cos(lon)/constants.astronomical_unit, 
             r*np.sin(lon)/constants.astronomical_unit, 
             'o', label=obj.name, markersize=8, 
             marker=euhforia.plot.marker.style[obj.name]['marker'], 
             color=euhforia.plot.marker.style[obj.name]['color'])

    ax1.legend(loc='lower center', bbox_to_anchor=(0.5, -0.18), 
               ncol=4, numpoints=1, fancybox=True,
               fontsize=7)

    if np.isclose(lon, meridional_slice):        
        ax2.plot(r*np.cos(lat)/constants.astronomical_unit, 
                 r*np.sin(lat)/constants.astronomical_unit, 
                 'o', label=obj.name, markersize=8, 
                 marker=euhforia.plot.marker.style[obj.name]['marker'], 
                 color=euhforia.plot.marker.style[obj.name]['color'])

plt.tight_layout()

#plt.savefig("pic", dpi=200)

The same functionality is included in the `euhforia.plot.slice.equatorial_and_meridional` function:

In [ ]:
fig, ax1, ax2 = euhforia.plot.slice.equatorial_and_meridional(data, variable="vr", meridional_slice=meridional_slice, 
                                              levels=np.linspace(200, 1600, 120),
                                              cmap=euhforia.plot.colormap.citrus,
                                              heliospheric_objects=heliospheric_objects,
                                              colorbar_ticks=np.linspace(200, 1600, 8));
ax2.set_xlim([0, 1]);
plt.tight_layout();

## Plotting a derived quantity

What about when one wants to plot a quantity that is not stored on the grid? For instance, how should one plot the scaled number density

\begin{equation}
n_\mathrm{scaled} = n(\mathbf{x}) \cdot \left(\frac{r}{1 \mathrm{AU}}\right)^2
\end{equation}

or temperature

\begin{equation}
T = \frac{P}{n k_B }
\end{equation}

? This can be done using the `add_variable`-method. First, compute the variable:

In [ ]:
T = data.P/(1e12*data.n*constants.kB)

Now, add it as a plottable variable:

In [ ]:
data.add_variable(T, name="T", unit="MK")

And plot:

In [ ]:
fig, ax1, ax2 = euhforia.plot.slice.equatorial_and_meridional(data, variable="T", meridional_slice=meridional_slice, 
                                              levels=np.linspace(0, 2, 120),
                                              cmap=euhforia.plot.colormap.citrus,
                                              heliospheric_objects=heliospheric_objects,
                                              colorbar_ticks=np.linspace(0, 2, 6));
ax2.set_xlim([0, 1]);
plt.tight_layout();

Similarly, for the scaled number density:

In [ ]:
nscaled = np.zeros(data.n.shape)

for idx, r in enumerate(data.grid.center_coords.r):
    nscaled[idx, :, :] = data.n[idx, :, :]*(r/constants.astronomical_unit)**2

In [ ]:
data.add_variable(nscaled, name="$n \, (r / 1 \mathrm{AU})^2$", unit="cm$^{-3}$")

In [ ]:
fig, ax1, ax2 = euhforia.plot.slice.equatorial_and_meridional(data, variable="$n \, (r / 1 \mathrm{AU})^2$", 
                                                              meridional_slice=meridional_slice, 
                                                              levels=np.linspace(0, 60, 120),
                                                              cmap=euhforia.plot.colormap.citrus,
                                                              heliospheric_objects=heliospheric_objects,
                                                              colorbar_ticks=np.linspace(0, 60, 7));
ax2.set_xlim([0, 1]);
plt.tight_layout();

## Time-sequence

A sequence of plots can be made by simply performing on of the plot operations above on several sets of files. First, the times at which there is output is determined:

In [ ]:
import glob
import re

In [ ]:
date_strs = []
for f in glob.glob("../../run/twin_cme_output/twin_cme_*.npz"):
    match = re.search(r'\d{4}-\d{2}-\d{2}T\d{2}-\d{2}-\d{2}', f)
    date_strs.append(match.group())

Then it is simply to plot one by one:

In [ ]:
for idx, date_str in enumerate(date_strs):
    
    # Load data
    data = euhforia.core.io.load_heliospheric_data("../../run/twin_cme_output/twin_cme_" + date_str + "*")
    
    # Plot
    fig, ax, cax = euhforia.plot.slice.equatorial(data, "vr", 
                                                  levels=np.linspace(200, 1600, 120),
                                                  cmap=euhforia.plot.colormap.citrus,
                                                  figsize=(15,8)); 
    
    # Add planets and SC
    for obj in (mercury, venus, earth, sta):
    
        r, lat, lon = obj.position(data.datetime)
    
        ax.plot(r*np.cos(lon)/constants.astronomical_unit, 
                r*np.sin(lon)/constants.astronomical_unit, 
                'o', label=obj.name, markersize=8, 
                marker=euhforia.plot.marker.style[obj.name]['marker'], 
                color=euhforia.plot.marker.style[obj.name]['color'])

    ax.legend(loc='lower center', bbox_to_anchor=(0.5, -0.2), ncol=4, numpoints=1, fancybox=True)
    
    
    # Save to file
    fig.savefig("test_" + str(idx).zfill(4) + ".png")

***